In [1]:
#import packages
import numpy as np
import gurobipy as gp
from gurobipy import*
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
df = pd.read_csv('paths_1219.csv', index_col=0)
df

,Combination,Shortest Path - Average Lanes per Unit Length,Shortest Path - Total Length,Second Shortest Path - Average Lanes per Unit Length,Second Shortest Path - Total Length,Shortest Path - Nodes,Second Shortest Path - Nodes,Third Shortest Path - Average Lanes per Unit Length,Third Shortest Path - Total Length,Relief Center 1,Relief Center 2
0,0,1.109753,1701.368,1.081249,2298.257,"[3854483397, 321565322, 2281072959, 321565347,...","[3854483397, 6240955569, 4406916636, 321565364...",2.007222,2779.171,1,2
1,1,1.000000,2109.426,1.061810,2154.893,"[3854483397, 3854483400, 3854483402, 390195492...","[3854483397, 321565322, 2281072959, 5203438334...",1.032073,2527.995,1,3
2,2,1.000000,2825.696,3.238514,3284.391,"[3854483397, 3854483400, 3854483402, 390195492...","[3854483397, 321565321, 3901954922, 253559471,...",1.065519,3505.746,1,4
3,3,1.976768,2338.724,1.056461,2601.906,"[3854483397, 321565321, 3901954922, 253559471,...","[3854483397, 3854483400, 3854483402, 390195492...",1.411252,2687.420,1,5
4,4,1.145909,3720.788,1.204740,4069.077,"[3854483397, 321565322, 2281072959, 321565347,...","[3854483397, 3854483400, 3854483402, 390195492...",1.371260,4158.834,1,6
...,...,...,...,...,...,...,...,...,...,...,...
67,67,1.000000,1034.257,1.000000,1638.329,"[655374965, 655374966, 8437939757, 3208478121,...","[655374965, 6982059059, 4554226138, 4554226137...",1.000000,1056.532,9,4
68,68,1.003375,2367.981,1.015669,2738.398,"[655374965, 6982059059, 4554226138, 4554226137...","[655374965, 655374966, 8437939757, 3208478121,...",1.156156,3190.257,9,5
69,69,1.137664,1635.601,1.114371,1844.752,"[655374965, 655374966, 8437939757, 3208478121,...","[655374965, 6982059059, 6252193599, 4510431306...",1.874907,1848.803,9,6
70,70,1.841209,3361.924,1.170005,3711.800,"[655374965, 655374966, 8437939757, 3208478121,...","[655374965, 6982059059, 6252193599, 4510431306...",1.765978,4662.954,9,7


In [2]:
def Traveling_Time_Damaged(table, PGD):

    from scipy.integrate import quad
    import math
    import numpy as np

    path_table = table.copy()

    m_main = 24
    beta_main = 0.7

    m_secondary = 12
    beta_secondary = 0.7

    def integrand_main(x):
        return 1/((2*math.pi)**(1/2)*beta_main*x) * math.e**(-1/2*((math.log(x) - math.log(m_main))/beta_main)**2)

    def integrand_sec(x):
        return 1/((2*math.pi)**(1/2)*beta_secondary*x) * math.e**(-1/2*((math.log(x) - math.log(m_secondary))/beta_secondary)**2)


    path_table['Time_1st'] = (path_table['Shortest Path - Total Length']/1000)/28.5 * 60
    path_table['Time_2nd'] = (path_table['Second Shortest Path - Total Length']/1000)/28.5 * 60

    lanes_median = np.median(list(path_table['Shortest Path - Average Lanes per Unit Length']) + list(path_table['Second Shortest Path - Average Lanes per Unit Length']))
    
    time_1st_ori = list(path_table['Time_1st'])
    lane_1st = list(path_table['Shortest Path - Average Lanes per Unit Length'])
    damaged_time_1st = []
    
    for i in range(path_table.shape[0]):
        if lane_1st[i] >= lanes_median:
            p, error = quad(integrand_main, 0, PGD)
            beta_prob_temp = np.random.beta(10 - 10*p, 10*p)
            damaged_time_1st.append(time_1st_ori[i]*(1 + beta_prob_temp))
        else:
            p, error = quad(integrand_sec, 0, PGD)        
            beta_prob_temp = np.random.beta(10 - 10*p, 10*p)
            damaged_time_1st.append(time_1st_ori[i]*(1+beta_prob_temp))


    time_2nd_ori = list(path_table['Time_2nd'])
    lane_2nd = list(path_table['Second Shortest Path - Average Lanes per Unit Length'])
    damaged_time_2nd = []
    
    for i in range(path_table.shape[0]):
        if lane_2nd[i] >= lanes_median:
            p, error = quad(integrand_main, 0, PGD)
            beta_prob_temp = np.random.beta(10 - 10*p, 10*p)
            damaged_time_2nd.append(time_2nd_ori[i]*(1 + beta_prob_temp))
        else:
            p, error = quad(integrand_sec, 0, PGD)        
            beta_prob_temp = np.random.beta(10 - 10*p, 10*p)
            damaged_time_2nd.append(time_2nd_ori[i]*(1+beta_prob_temp))

    path_table['damaged_time_1st'] = damaged_time_1st
    path_table['damaged_time_2nd'] = damaged_time_2nd

    node_1 = path_table['Relief Center 1']
    node_2 = path_table['Relief Center 2']
    damaged_time_1 = path_table['damaged_time_1st']
    damaged_time_2 = path_table['damaged_time_2nd']
    node_num = max(node_1)

    time_table = []
    for i in range(node_num):
        temp = []
        for j in range(node_num):
            if i == j:
                temp.append(0)
            else:
                for k in range(path_table.shape[0]):
                    if node_1[k] == i+1 and node_2[k] == j+1:
                        temp.append(min(damaged_time_1[k], damaged_time_2[k]))
                        break
        time_table.append(temp)

    time_dict = {}
    for i in range(node_num):
        for j in range(node_num):
            time_dict[(i+1, j+1)] = time_table[i][j]

    display(path_table)

    return time_dict


In [3]:
def Traveling_Time_Damaged_(table, PGD):

    from scipy.integrate import quad
    import math
    import numpy as np

    path_table = table.copy()

    m_main = 24
    beta_main = 0.7

    m_secondary = 12
    beta_secondary = 0.7

    def integrand_main(x):
        return 1/((2*math.pi)**(1/2)*beta_main*x) * math.e**(-1/2*((math.log(x) - math.log(m_main))/beta_main)**2)

    def integrand_sec(x):
        return 1/((2*math.pi)**(1/2)*beta_secondary*x) * math.e**(-1/2*((math.log(x) - math.log(m_secondary))/beta_secondary)**2)

    speed = 28.5
    path_table['Time_1st'] = (path_table['Shortest Path - Total Length']/1000)/speed * 60
    path_table['Time_2nd'] = (path_table['Second Shortest Path - Total Length']/1000)/speed * 60
    path_table['Time_3rd'] = (path_table['Third Shortest Path - Total Length']/1000)/speed * 60

    lanes_median = np.median(list(path_table['Shortest Path - Average Lanes per Unit Length']) + list(path_table['Second Shortest Path - Average Lanes per Unit Length']) + list(path_table['Third Shortest Path - Average Lanes per Unit Length']))
    
    time_1st_ori = list(path_table['Time_1st'])
    lane_1st = list(path_table['Shortest Path - Average Lanes per Unit Length'])
    damaged_time_1st = []
    
    for i in range(path_table.shape[0]):
        if lane_1st[i] >= lanes_median:
            p, error = quad(integrand_main, 0, PGD)
            if p >= 0.99:
                p = 0.99
            if p <= 0.01:
                p = 0.01
            beta_prob_temp = np.random.beta(10 - 10*p, 10*p)
            damaged_time_1st.append(time_1st_ori[i]*(1 + beta_prob_temp))
        else:
            p, error = quad(integrand_sec, 0, PGD)   
            if p >= 0.99:
                p = 0.99
            if p <= 0.01:
                p = 0.01     
            beta_prob_temp = np.random.beta(10 - 10*p, 10*p)
            damaged_time_1st.append(time_1st_ori[i]*(1+beta_prob_temp))


    time_2nd_ori = list(path_table['Time_2nd'])
    lane_2nd = list(path_table['Second Shortest Path - Average Lanes per Unit Length'])
    damaged_time_2nd = []
    
    for i in range(path_table.shape[0]):
        if lane_2nd[i] >= lanes_median:
            p, error = quad(integrand_main, 0, PGD)
            if p >= 0.99:
                p = 0.99
            if p <= 0.01:
                p = 0.01
            beta_prob_temp = np.random.beta(10 - 10*p, 10*p)
            damaged_time_2nd.append(time_2nd_ori[i]*(1 + beta_prob_temp))
        else:
            p, error = quad(integrand_sec, 0, PGD)  
            if p >= 0.99:
                p = 0.99
            if p <= 0.01:
                p = 0.01      
            beta_prob_temp = np.random.beta(10 - 10*p, 10*p)
            damaged_time_2nd.append(time_2nd_ori[i]*(1+beta_prob_temp))

    time_3rd_ori = list(path_table['Time_3rd'])
    lane_3rd = list(path_table['Third Shortest Path - Average Lanes per Unit Length'])
    damaged_time_3rd = []
    
    for i in range(path_table.shape[0]):
        if lane_3rd[i] >= lanes_median:
            p, error = quad(integrand_main, 0, PGD)
            if p >= 0.99:
                p = 0.99
            if p <= 0.01:
                p = 0.01
            beta_prob_temp = np.random.beta(10 - 10*p, 10*p)
            damaged_time_3rd.append(time_3rd_ori[i]*(1 + beta_prob_temp))
        else:
            p, error = quad(integrand_sec, 0, PGD)        
            if p >= 0.99:
                p = 0.99
            if p <= 0.01:
                p = 0.01
            beta_prob_temp = np.random.beta(10 - 10*p, 10*p)
            damaged_time_3rd.append(time_3rd_ori[i]*(1+beta_prob_temp))

    path_table['damaged_time_1st'] = damaged_time_1st
    path_table['damaged_time_2nd'] = damaged_time_2nd
    path_table['damaged_time_3rd'] = damaged_time_3rd

    node_1 = path_table['Relief Center 1']
    node_2 = path_table['Relief Center 2']
    damaged_time_1 = path_table['damaged_time_1st']
    damaged_time_2 = path_table['damaged_time_2nd']
    damaged_time_3 = path_table['damaged_time_3rd']
    node_num = max(node_1)

    time_table = []
    for i in range(node_num):
        temp = []
        for j in range(node_num):
            if i == j:
                temp.append(0)
            else:
                for k in range(path_table.shape[0]):
                    if node_1[k] == i+1 and node_2[k] == j+1:
                        temp.append(min(damaged_time_1[k], damaged_time_2[k], damaged_time_3[k]))
                        break
        time_table.append(temp)

    time_dict = {}
    for i in range(node_num):
        for j in range(node_num):
            time_dict[(i+1, j+1)] = time_table[i][j]

    # display(path_table)

    return time_dict


In [4]:
vehicle_num = 3
period_num = 20
node_num = 7
commodity_num = 2
carryingVolume = 250
iteration_num = 1

H = [h+1 for h in range(vehicle_num)] ##V
PI = [pi+1 for pi in range(period_num)] ## period
N = [n+1 for n in range(node_num)] ## node
G = [g+1 for g in range(commodity_num)] ## commodity


I = {(1, 1): 999999, (1, 2): 30, (1, 3): 20, (1, 4): 50, (1, 5): 30, (1, 6): 40
     (2, 1): 999999, (2, 2): 20, (2, 3): 30, (2, 4): 20, (2, 5): 30, (2, 6): 20} # (commodity, node): inventory 
Q = {(1, 1): 0, (1, 2): 60, (1, 3): 50, (1, 4): 90, (1, 5): 50, (1, 6): 70,
     (2, 1): 0, (2, 2): 40, (2, 3): 50, (2, 4): 30, (2, 5): 40, (2, 6): 40} # (commodity, node): demand

volume = {1: 5, 2: 10}  ## volume per unit for each commodity

OBJ_list = []
X_list = []
Y_list = []
for S in range(iteration_num):

    print('\n ### iteration ' + str(S+1) + ' ### \n')

    # t = {(1, 1):0, (1, 2):20, (1, 3):40, (1, 4):35, (1, 5):20, 
    # (2, 1):45, (2, 2):0, (2, 3):30, (2, 4):50, (2, 5):30, 
    # (3, 1):20, (3, 2):15, (3, 3):0, (3, 4):10, (3, 5):20, 
    # (4, 1):45, (4, 2):30, (4, 3):55, (4, 4):0, (4, 5):60, 
    # (5, 1):20, (5, 2):10, (5, 3):30, (5, 4):25, (5, 5):0}  # travel time between nodes

    t = Traveling_Time_Damaged_(df, 100)

    #Create a new model
    m = Model("Multi-Commodity Distribution")
    # m.setParam('OutputFlag', 0)

    #Decision Variables
    x = {}
    for h in H:
        for pi in PI:
            for a in N:
                for b in N:
                    if a != b:
                        x[h, pi, a, b] = m.addVar(vtype=GRB.BINARY, name=f"x_vehicle_{h}_journey_{pi}_form_{a}_to_{b}")
                    else:
                        x[h, pi, a, b] = 0

    y = {}
    for h in H:
        for pi in PI:
            for g in G:
                for a in N:
                    for b in N:
                        if a != b:
                            y[h, pi, g, a, b] = m.addVar(vtype=GRB.CONTINUOUS, name=f"y_vehicle_{h}_journey_{pi}_good_{g}_form_{a}_to_{b}")
                        else:
                            y[h, pi, g, a, b] = 0

    T = m.addVar(vtype=GRB.CONTINUOUS, name='MaxToatalTime')

    #Objective Function: Minimize the maximum completion time of all vehicle journeys
    # m.setObjective(quicksum(x[h,pi,a,b] * t[(a,b)] for h in H for pi in PI for a in N for b in N ), GRB.MINIMIZE)
    m.setObjective(T, GRB.MINIMIZE)


    #Constraint 1: a given vehicle will only use one link to move in each journey
    for h in H:
        for pi in PI:
            m.addConstr(quicksum(x[h,pi,a,b] for a in N for b in N) <= 1, f"vehicle_{h}_journey_{pi}_link")

    #Constraint 2: allow vehicle journeys to be connected
    for h in H:
        for pi in PI:
            if pi > 1: #do not want apply this constraint for the first journey; there is no previous journey
                for b in N: 
                    previousLink = quicksum(x[h,pi-1,a,b] for a in N)
                    afterLinks = quicksum(x[h,pi,b,c] for c in N)
                    m.addConstr(afterLinks <= previousLink, f"vehicle_{h}_node_{b}_journey_continuity_{pi}")

    # Constraint 3: the commodities shipped to DC is zero
    for g in G:
        for pi in PI:
            leftEquation1 = quicksum(y[h, pi, g, a, 1] for h in H for a in N if (g, 1) in Q and (h, pi, g, a, 1) in y)  # Sum of quantities shipped to DC for each commodity
            m.addConstr(leftEquation1 == 0, f"DC_commodity_{g}_shipment_{pi}")


                
    #Constraint 4: final inventory level of all nodes must be greater than or equal to their demand.
    for g in G:
        for b in N:
            # Check if the key exists in both dictionaries
            if (g, b) in I and (g, b) in Q:
                incomingInventory = quicksum(y[h, pi, g, a, b] for h in H for pi in PI for a in N)
                outcomingInventory = quicksum(y[h, pi, g, b, c] for h in H for pi in PI for c in N)
                m.addConstr(I[g, b] + incomingInventory - outcomingInventory >= Q[g, b], f"inventory_node_{b}_commodity_{g}")


    #Cosntraint 5: vehicle capacity
    for h in H:
        for pi in PI:
            for a in N:
                for b in N:
                    leftEquation2 = quicksum(y[h,pi,g,a,b]*volume[g] for g in G )
                    rightEquation2 = carryingVolume * x[h,pi,a,b]
                    m.addConstr(leftEquation2 <= rightEquation2, f"vehicle_{h}_capacity_link_{a}_{b}_journey_{pi}")

    #Constraint 6: Maximun Total time
    for h in H:
        m.addConstr(T >= quicksum(x[h,pi,a,b] * t[(a,b)] for pi in PI for a in N for b in N))

    #Optimize the model
    m.setParam('Timelimit', 60 * 60)
    m.optimize()

    #Output Results
    # if m.status == GRB.OPTIMAL:
    #     for v in m.getVars():
    #         print(f'{v.varName}: {v.x}')

    #Print the objective value
    if m.status == GRB.OPTIMAL:
        print(f'Optimal Objective: {m.objVal}')
        OBJ_list.append(m.objVal)
    else:
        OBJ_list.append('not optimal')
    
    x_temp = {}
    for h in H:
        for pi in PI:
            for a in N:
                for b in N:
                    if a != b:
                        x_temp[h, pi, a, b] = x[h, pi, a, b].x
                    else:
                        x_temp[h, pi, a, b] = 0
    X_list.append(x_temp)

    y_temp = {}
    for h in H:
        for pi in PI:
            for g in G:
                for a in N:
                    for b in N:
                        if a != b:
                            y_temp[h, pi, g, a, b] = y[h, pi, g, a, b].x
                        else:
                            x_temp[h, pi, g, a, b] = 0
    Y_list.append(y_temp)



 ### iteration 1 ### 

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-18
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 3456 rows, 7561 columns and 28191 nonzeros
Model fingerprint: 0xdc68f9a4
Variable types: 5041 continuous, 2520 integer (2520 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 822 rows and 753 columns
Presolve time: 0.03s
Presolved: 2634 rows, 6808 columns, 24129 nonzeros
Variable types: 4321 continuous, 2487 integer (2487 binary)

Root relaxation: objective 2.264534e+01, 4094 iterations, 0.12 seconds (0.28 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

In [5]:
print(OBJ_list)

['not optimal']


In [ ]:
for s in range(iteration_num):
    print(f'# iteration_{s+1}')
    for h in H:
        print(f'## car_{h}')
        for pi in PI:
            for a in N:
                for b in N:
                    if X_list[s][h, pi, a, b] > 0:
                        for g in G:
                            print(f'y[{h}, {pi}, {g}, {a}, {b}] = {Y_list[s][h, pi, g, a, b]}')
    print('\n')

In [2]:
temp = [20.365358283245854, 19.772822009427365, 19.929591172516442, 19.77537279030891, 20.038124020132692, 19.807933390995085, 19.773318685072518, 19.773914566953415, 19.93884690224053, 19.84501093965277, 19.808434826780434, 19.794588748795853, 19.89298633084208, 19.77432060235203, 20.348245120735307, 19.80225644491412, 20.099136371953442, 20.409888826975592, 20.388736721320967, 19.777012755018056, 19.990797689913038, 19.78023502253152, 19.776889540563495, 20.107241758761965, 19.78233183805824, 19.806165061087196, 20.32690448081895, 19.77829504636913, 19.777208929747708, 19.77268652241893]

In [3]:
import numpy as np

In [4]:
np.mean(temp)

19.933821846683454

In [5]:
np.var(temp)

0.046999822155153456